In [1]:
import json
from datetime import datetime
import pandas as pd
from openai_query import consultar_chatgpt
import duckdb as ddbb
from datetime import datetime, timedelta

def get_d_minus_1_timestamps_unix():
    # Data e hora atual
    hoje = datetime.now()
    
    # Data de D-1
    d1 = hoje - timedelta(days=1)
    
    # Limites do dia D-1 (00:00:00 e 23:59:59)
    d1_min = d1.replace(hour=0, minute=0, second=0, microsecond=0)
    d1_max = d1.replace(hour=23, minute=59, second=59, microsecond=999999)
    
    # Converte para timestamp UNIX (inteiros)
    ts_min = int(d1_min.timestamp())
    ts_max = int(d1_max.timestamp())
    
    # Data atual no formato yyyymmdd
    data_atual = hoje.strftime("%Y%m%d")
    
    return data_atual, ts_min, ts_max

# Exemplo de uso:
data_atual, ts_min, ts_max = get_d_minus_1_timestamps_unix()

In [50]:
con = ddbb.connect()
con.install_extension('json')
con.load_extension('json')
con.create_function('consultar_chatgpt', consultar_chatgpt)
con.execute("INSTALL excel;")
con.execute("LOAD excel;")

In [5]:
#CAMADA BRONZE
path = r'landing\*.json'
df = con.execute('''
    SELECT
        *
    FROM read_json_auto(?)
''', [path]).df()

df.to_parquet(f'bronze/bronze_messages_{data_atual}.parquet', index=False, compression='snappy')

In [ ]:
#CAMADA SILVER
path = r'bronze\*.parquet'
df = con.execute('''
    SELECT DISTINCT
        id,
        strftime(to_timestamp(cast(timestamp as bigint)), '%Y-%m-%d') as date,
        strftime(to_timestamp(cast(timestamp as bigint)), '%H:%M:%S') as time,
        substring((split(cast(_data.Info.SenderAlt as varchar),':'))[1], 0, 3) as country_code,
        substring((split(cast(_data.Info.SenderAlt as varchar),':'))[1], 3, 2) as state_code,
        substring((split(cast(_data.Info.SenderAlt as varchar),':'))[1], 5, 9).replace('@', '') as tel_number,
        body.replace('\n', ' ').replace('\r', ' ') as body,
        cast(_data.Message.extendedTextMessage.text as varchar).replace('\n', ' ').replace('\r', ' ') as text,
        cast(_data.Message.extendedTextMessage.title as varchar).replace('\n', ' ').replace('\r', ' ') as title,
        cast(_data.Message.extendedTextMessage.description as varchar).replace('\n', ' ').replace('\r', ' ') as description,
        cast(_data.Message.imageMessage.caption as varchar).replace('\n', ' ').replace('\r', ' ') as caption
                 
    FROM read_parquet(?)
    WHERE (body like '%http%' OR body like '%www.%')
    AND (body not like '%whatsapp%')
''', [path]).df()

df.to_parquet(f'silver/silver_messages_{data_atual}.parquet', index=False, compression='snappy')

In [ ]:
#CAMADA GOLD
path = r'silver\*.parquet'
groups = r'groups.json'
df = con.execute('''
    WITH parte1 as (
    SELECT DISTINCT
        silver.id, silver.date, silver.time, silver.country_code, silver.state_code, silver.tel_number, silver.body, silver.text, silver.title, silver.description, silver.caption,
        case
        -- ECOMMERCE
        when lower(body) like '%mercadolivre%' or lower(text) like '%mercadolivre%' or lower(title) like '%mercadolivre%' or lower(description) like '%mercadolivre%' or lower(caption) like '%mercadolivre%' then 'MERCADO LIVRE'
        when lower(body) like '%magalu%' or lower(text) like '%magalu%' or lower(title) like '%magalu%' or lower(description) like '%magalu%' or lower(caption) like '%magalu%' then 'MAGAZINE LUIZA'
        when lower(body) like '%shopee%' or lower(text) like '%shopee%' or lower(title) like '%shopee%' or lower(description) like '%shopee%' or lower(caption) like '%shopee%' then 'SHOPEE'
        when lower(body) like '%amzn%' or lower(text) like '%amzn%' or lower(title) like '%amzn%' or lower(description) like '%amzn%' or lower(caption) like '%amzn%' then 'AMAZON'
        when lower(body) like '%amazon%' or lower(text) like '%amazon%' or lower(title) like '%amazon%' or lower(description) like '%amazon%' or lower(caption) like '%amazon%' then 'AMAZON'
        when lower(body) like '%shein%' or lower(text) like '%shein%' or lower(title) like '%shein%' or lower(description) like '%shein%' or lower(caption) like '%shein%' then 'SHEIN'
        -- AGREGADORES
        when lower(body) like '%achadosprincipais%' or lower(text) like '%achadosprincipais%' or lower(title) like '%achadosprincipais%' or lower(description) like '%achadosprincipais%' or lower(caption) like '%achadosprincipais%' then 'AGREGADOR'
        when lower(body) like '%promorelampago%' or lower(text) like '%promorelampago%' or lower(title) like '%promorelampago%' or lower(description) like '%promorelampago%' or lower(caption) like '%promorelampago%' then 'AGREGADOR'
        when lower(body) like '%minhaloja.%' or lower(text) like '%minhaloja.%' or lower(title) like '%minhaloja.%' or lower(description) like '%minhaloja.%' or lower(caption) like '%minhaloja.%' then 'AGREGADOR'
        -- BETS
        when lower(body) like '%aa7.%' or lower(text) like '%aa7.%' or lower(title) like '%aa7.%' or lower(description) like '%aa7.%' or lower(caption) like '%aa7.%' then 'BETS'
        when lower(body) like '%bmw7.%' or lower(text) like '%bmw7.%' or lower(title) like '%bmw7.%' or lower(description) like '%bmw7.%' or lower(caption) like '%bmw7.%' then 'BETS'
        else 'OUTROS' end as origin,
        case
        when lower(body) like '%cupom%' or lower(text) like '%cupom%' or lower(title) like '%cupom%' or lower(description) like '%cupom%' or lower(caption) like '%cupom%' then True
        else False end as coupon,
        coalesce(groups.name, 'N/A') as group_name
    FROM read_parquet(?) as silver
    LEFT JOIN read_json_auto(?) AS groups
    ON (split(silver.id, '_'))[2] = groups.id
    )
                 
    SELECT id, date, time, country_code, state_code, tel_number, origin, coupon, group_name,
    case
    when origin <> 'BETS' then consultar_chatgpt(cast(concat(coalesce(body, 'N/A'), ' - ', coalesce(text, 'N/A'), ' - ', coalesce(title, 'N/A'), ' - ', coalesce(description, 'N/A'), ' - ', coalesce(caption, 'N/A')) as varchar))
    else 'N/A' end as category
    FROM parte1
''', [path, groups]).df()

# df.head(2)
df.to_parquet(f'gold/gold_messages_{data_atual}.parquet', index=False, compression='snappy')
df.to_csv(f'gold/gold_messages_{data_atual}.csv', index=False)

,id,date,time,country_code,state_code,tel_number,origin,coupon,group_name,category
0,false_120363255308049173@g.us_2AF00AFFAEACBFD0...,2025-11-04,15:12:26,55,11,986255386,MERCADO LIVRE,True,PROMO LOVERS 🛍️ #14,Beleza e Perfumaria
1,false_120363257853840248@g.us_3EB0E7873C8FFB48...,2025-11-04,15:32:45,39,37,92555853,MERCADO LIVRE,True,ACHADINHO 🛍️ #16,Automotivo
